In [ ]:
import os, yaml, subprocess, glob, math

import pprint, h5py
import numpy as np
import scipy as sp
import pandas as pd
import nibabel as nb
import cortex as cx
import seaborn as sns

import matplotlib.pyplot as plt
from nilearn.glm.first_level.hemodynamic_models import _gamma_difference_hrf

In [ ]:
base_dir = '/Users/knapen/projects/soma_visual/data/derivatives/fmriprep/'
beh_dir = '/Users/knapen/projects/soma_visual/data/sourcedata_nordic/'
op_dir = '/Users/knapen/projects/soma_visual/data/derivatives/glms/'

subject = 'sub-06'
# visual-motor, visual-only, and auditory-motor
conditions = ['VM', 'VO', 'AM']

nr_trs_dropped = 4
nr_acompcorrs = 4
exclude_bilateral_regressors = True

space_string = 'space-fsLR_den-170k_bold.dtseries.nii'

runs = [glob.glob(os.path.join(base_dir, subject, 'func', f'{subject}_task-{condition}*{space_string}')) for condition in conditions]
for run in runs:
    run.sort()
    if len(run) > 0:
        run.pop(0)
# pprint.pprint(runs)

confound_runs = [glob.glob(os.path.join(base_dir, subject, 'func', f'{subject}_task-{condition}_*desc-confounds_timeseries.tsv')) for condition in conditions]
for run in confound_runs:
    run.sort()
    if len(run) > 0:
        run.pop(0)
# pprint.pprint(confound_runs)

event_runs = [glob.glob(os.path.join(beh_dir, subject, 'func', f'{subject}_*_task-{condition}_events.tsv')) for condition in conditions]
for run in event_runs:
    run.sort()
    if len(run) > 0:
        run.pop(0)
# pprint.pprint(event_runs)

In [ ]:
this_condition = 'VO'

# read data, drop first 4 TRs which were dummies
single_timecourses = np.array([nb.load(run).get_fdata()[nr_trs_dropped:] for run in runs[conditions.index(this_condition)]])
single_timecourses -= sp.signal.savgol_filter(single_timecourses, window_length=single_timecourses.shape[1]-1, polyorder=2, axis=1)
single_mean_epi = single_timecourses[:,5:15].mean(1)[:,np.newaxis,:]
single_psc_data = np.nan_to_num(100 * (single_timecourses - single_mean_epi) / single_mean_epi)

# average across runs
psc_data = np.median(single_psc_data, axis=0)

tr = 1.6
n_timepoints = psc_data.shape[0]
tr_timepoints = np.arange(tr/2,n_timepoints*tr, tr)

In [ ]:
event_dfs = [pd.read_csv(run, sep='\t') for run in event_runs[conditions.index(this_condition)]]

confound_dfs = [pd.read_csv(run, sep='\t')[nr_trs_dropped:].set_index(tr_timepoints) for run in confound_runs[conditions.index(this_condition)]]
confound_dfs_z = [(cdf-cdf.mean(0))/cdf.std(0) for cdf in confound_dfs]

f, axs = plt.subplots(1, len(confound_dfs), figsize=(20,5))
for i in range(len(confound_dfs)):
    sns.heatmap(confound_dfs_z[i], ax=axs[i], vmin=-8, vmax=8, cmap='inferno')

In [ ]:
confound_columns = ['cosine00'] + [f'a_comp_cor_{str(i).zfill(2)}' for i in range(nr_acompcorrs)]

confound_dms = [cdfz[confound_columns] for cdfz in confound_dfs_z]

In [ ]:
plt.imshow(confound_dms[0].T)

In [ ]:
def get_relevant_event_data(edf):
    ts = edf.response == 't'
    start_time = edf[ts].onset_abs.values[0]

    trials = ~edf.stimulus.isnull()
    stims = edf.event_type == 'stim'
    stim_trials = edf[trials & stims]
    stim_trials['net_onsets'] = stim_trials.onset_abs-start_time

    return stim_trials

event_data = [get_relevant_event_data(edf) for edf in event_dfs]

onsets = np.array([ed.net_onsets.values for ed in event_data])
plt.plot(onsets.var(0), label='timing variance across runs')
plt.plot(np.ones(onsets.shape[1])*1/120, label='display frame times')
plt.xlabel('trials')
plt.ylabel('duration [s]');
plt.legend()

np.unique(event_data[0].stimulus)

for edf in event_data:
    edf['mean_onset'] = onsets.mean(0)

In [ ]:
unique_event_types = ['eyebrows', 'eyes', 'mouth', 'tongue', 'lhand_fing1', 'lhand_fing2', 'lhand_fing3', 'lhand_fing4', 'lhand_fing5', 'lleg',  'rhand_fing1', 'rhand_fing2', 'rhand_fing3',
 'rhand_fing4', 'rhand_fing5', 'rleg', 'bhand_fing1', 'bhand_fing2', 'bhand_fing3', 'bhand_fing4', 'bhand_fing5', 'bleg']

In [ ]:
# okay, now that we have event times, we'll be able to create a fine-timing timecourse. 
# I will tell you a secret: the TR in this experiment was 1.6s - you can find this info
# in the figshare website also. 


def create_BOLD_regressor(event_times, n_timepoints, tr=1.6, supersampleratio=10):

    # create 0.1 s timescale regressor, and fill in the events
    upscaled_times = np.arange(0, n_timepoints*tr, 1/supersampleratio)
    neural_event_timecourse = np.zeros(int(n_timepoints*tr*supersampleratio))
    # we'll pretend the events were instantaneous, since this will not
    # impact the expected response shapes much. For events with variable durations
    # you'll want to make sure this doesn't happen :-)
    neural_event_timecourse[np.round(event_times * supersampleratio).astype(int)] = 1 

    hrf = _gamma_difference_hrf(tr=tr ,oversampling=tr*supersampleratio, onset=-tr/2)
    hrf /= hrf.max() # <- normalize HRF

    bold_event_timecourse = np.convolve(neural_event_timecourse, hrf, 'full')[:neural_event_timecourse.shape[0]]
    sub_sampled_regressor = bold_event_timecourse[::int(tr*supersampleratio)]
    return sub_sampled_regressor

def create_design_matrix(regressor_types, expt_df, data):
    regressors = [np.ones(data.shape[1])]
    for regressor_type in regressor_types:
        event_times = np.array(expt_df[expt_df['stimulus'] == regressor_type]['mean_onset'])
        regressors.append(create_BOLD_regressor(event_times, data.shape[1]))
    return np.array(regressors)

def run_glm(run_data, run_dm):
    betas, _, _, _ = np.linalg.lstsq(run_dm, run_data)
    betas = pd.DataFrame(betas.T, columns=run_dm.columns)

    yhat = pd.DataFrame(np.dot(betas, run_dm.T).T, index=run_dm.index)
    errors =  pd.DataFrame((yhat.T-run_data.T).T, index=run_dm.index)
    
    sse = np.sum(np.nan_to_num(errors) ** 2, axis=0)
    rsq = 1-errors.var(axis=0)/run_data.var(axis=0)

    sse_rsq = pd.DataFrame(np.array([sse, rsq]).T, columns=['sse', 'rsq'])
    
    return betas, sse_rsq, yhat, errors

dm = create_design_matrix(regressor_types=unique_event_types, expt_df=event_data[0], data=psc_data.T)

In [ ]:
for x in range(1,6):
    to_add = dm[unique_event_types.index(f'bhand_fing{x}')+1]
    dm[unique_event_types.index(f'lhand_fing{x}')+1] += to_add
    dm[unique_event_types.index(f'rhand_fing{x}')+1] += to_add

# leg is only one
to_add = dm[unique_event_types.index('bleg')+1]
dm[unique_event_types.index('lleg')+1] += to_add
dm[unique_event_types.index('rleg')+1] += to_add

print(dm.shape)
sns.heatmap(dm)

if exclude_bilateral_regressors:
#     update dm & unique event types list to exclude the bilateral responses now
    dm = dm[:-6]
    unique_event_types = unique_event_types[:-6]

dm_df = pd.DataFrame(dm.T, columns=np.r_[['intercept'], unique_event_types], index=tr_timepoints)
sns.heatmap(dm_df)


dm_df_confs = [dm_df for cdm in confound_dms]

# dm_df_confs = [pd.concat([dm_df, cdm], axis=1) for cdm in confound_dms]
# f, axs = plt.subplots(1, len(dm_df_confs), figsize=(20,5))
# for i in range(len(dm_df_confs)):
#     sns.heatmap(dm_df_confs[i], ax=axs[i], vmin=-8, vmax=8, cmap='inferno')

In [ ]:
for run in range(len(runs[conditions.index(this_condition)])):
    run_data = pd.DataFrame(single_psc_data[run], index=tr_timepoints)
    run_dm = dm_df_confs[run]
        
    betas, sse_rsq, yhat, errors = run_glm(run_data, run_dm)
    
    opf_base = os.path.join(op_dir, subject, 'func', os.path.split(runs[conditions.index(this_condition)][run])[1].replace('.dtseries.nii', ''))
    
    betas.to_hdf(opf_base + '_betas.h5', key='betas', complevel=6)
    sse_rsq.to_hdf(opf_base + '_sse_rsq.h5', key='sse_rsq', complevel=6)
    yhat.to_hdf(opf_base + '_yhat.h5', key='yhat', complevel=6)    
    errors.to_hdf(opf_base + '_errors.h5', key='errors', complevel=6)
    run_dm.to_hdf(opf_base + '_dm.h5', key='run_dm', complevel=6)

In [ ]:
# rhands = np.array([1 if 'rhand' in uet else 0 for uet in unique_event_types])
# lhands = np.array([-1 if 'lhand' in uet else 0 for uet in unique_event_types])
# contrast_vector = np.r_[0, rhands+lhands] # the intercept regressor was never in the event types list

# c_betas = np.dot(contrast_vector, betas)

# N = mean_timecourses.shape[0]
# P = betas.shape[0]
# df = (N - P)

# sigma_hat = np.sum((mean_timecourses.T - yhat) ** 2, axis=1) / df
# des_var = contrast_vector.dot(np.linalg.pinv(dm.dot(dm.T))).dot(contrast_vector.T)

# t = c_betas / np.sqrt(sigma_hat * des_var)
# # t[np.abs(t) < 4] = t[np.abs(t) < 4]/4

# surf_t = np.r_[t[li], t[ri]]


# f2 = cx.quickshow(cx.Vertex(surf_t, subject='hcp_999999', cmap='CyanBlueGrayRedPink', vmin=-10, vmax=10), height=2048);
# # f2.suptitle('left hand vs right hand contrast');